In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.font_manager as fm
import japanize_matplotlib
import os
import re
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import pickle
import os


def load_tfidf_model(path, part):
    """
    指定されたパートに基づいて保存されたTF-IDFモデルを読み込み、ベクトルライザーを返す。

    Args:
    - path: モデルが保存されたディレクトリのパス
    - part: モデルが関連付けられたパート

    Returns:
    - vectorizer: 読み込まれたTF-IDFベクトルライザー
    """
    model_filename = os.path.join(path, f'tfidf_model_part{part}.pkl')

    try:
        with open(model_filename, 'rb') as file:
            vectorizer = pickle.load(file)
        print(f"TF-IDFモデルが'{model_filename}'から読み込まれました。")
        return vectorizer
    except Exception as e:
        print(f"Error during loading the model from '{model_filename}': {e}")
        return None

def train_tfidf_model(path, part):
    # 正規表現パターンの調整（Kana.tokファイルも含む）
    pattern = r'U[A-Z]{2}[0-9]{3}.[1-5].' + str(part) + r'(\.tok|Kana\.tok)'

    if not os.path.isdir(path):
        print(f"Error: '{path}' is not a valid directory.")
        return None

    documents = []
    for filename in os.listdir(path):
        if re.match(pattern, filename):
            try:
                with open(os.path.join(path, filename), 'r', encoding='utf-8') as file:
                    document = []
                    for line in file:
                        words = re.split('\t|\s', line.strip())
                        document.extend(words)
                    documents.append(' '.join(document))
            except Exception as e:
                print(f"Error reading file {filename}: {e}")

    if not documents:
        print("No documents found for the specified part. Exiting function.")
        return None

    print(f"Documents for part {part}:", documents)

    try:
        vectorizer = TfidfVectorizer()
        vectorizer.fit(documents)
        model_filename = f'tfidf_model_part{part}.pkl'
        with open(model_filename, 'wb') as file:
            pickle.dump(vectorizer, file)
        print(f"TF-IDFモデルがトレーニングされ、'{model_filename}'として保存されました。")
    except Exception as e:
        print(f"Error during training or saving the model: {e}")
        return None

    return vectorizer


# word_listから重要な単語リストを生成する関数
def generate_important_word_list(model, word_list):
    # word_listの単語をスペース区切りの文書として扱う
    document = " ".join(word_list)
    # トレーニング済みモデルを使用してTF-IDF変換
    tfidf_matrix = model.transform([document])
    tfidf_scores = tfidf_matrix.toarray().flatten()
    # 単語とそのスコアを辞書に格納
    feature_names = model.get_feature_names_out()
    word_scores = {word: tfidf_scores[idx] for idx, word in enumerate(feature_names) if word in word_list}
    #print("word_scores:", word_scores)
    # スコアに基づいて単語をソート
    sorted_words = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)
    # 上位10%の単語を選出
    top_10_percent = int(len(sorted_words) * 0.1)
    important_word_list = [word for word, score in sorted_words[:top_10_percent]]
    #print("important_word_list:", important_word_list)
    # TF-IDFモデルにない単語を追加
    for word in word_list:
        if word not in feature_names:
            important_word_list.append(word)

    important_word_list = list(set(important_word_list))
    #print("important_word_list:", important_word_list)
    return important_word_list


# 類似度が閾値以上の単語リストを作成し、グラフに追加する関数
def extend_graph(wv, G, word_list, graph_word_list, processed_words, similarity_cache, sth, N, ih):
    new_graph_word_list = set()

    for wi in graph_word_list:
        if wi in processed_words:
            continue  # 既に処理された単語はスキップ

        over_sth_word_list = []

        for wj in word_list:
            if wi != wj:
                # 類似度のキャッシュをチェック
                if (wi, wj) in similarity_cache:
                    similarity = similarity_cache[(wi, wj)]
                else:
                    try:
                        similarity = wv.similarity(wi, wj) if wi in wv.key_to_index and wj in wv.key_to_index else ih
                        similarity_cache[(wi, wj)] = similarity  # 類似度をキャッシュに保存
                    except KeyError:
                        similarity = ih  # エラーが発生した場合のデフォルト類似度

                if similarity >= sth:
                    over_sth_word_list.append(wj)

        if len(over_sth_word_list) >= N:
            G.add_node(wi)
            for wh in over_sth_word_list[:N]:
                G.add_edge(wi, wh)
            new_graph_word_list.update(over_sth_word_list[:N])

        #print("wi, over_sth_word_list:", wi, over_sth_word_list)
        processed_words.add(wi)

    return new_graph_word_list, processed_words, similarity_cache


def analyzeNetwork(tf_idf_vectorizer, word_list, sth, N, ih):
    """
    ネットワーク解析を行い、特徴量を計算する関数。

    Args:
    - tf_idf_vectorizer: TF-IDFベクトルライザー（モデル）
    - word_list: 解析対象の単語リスト
    - sth: 特定の閾値またはパラメータ
    - N: ネットワーク拡張のためのノード数の閾値
    - ih: 重要な単語を識別するための閾値またはパラメータ

    Returns:
    - features: 計算された特徴量の辞書
    """
    try:
        # important_word_listの作成
        important_word_list = generate_important_word_list(model=tf_idf_vectorizer, word_list=word_list)
        #print("important_word_list:", important_word_list)

        # ネットワークグラフの作成
        G = nx.Graph()

        # キーワードは最初にグラフノードとして設定
        for wi in important_word_list:
            G.add_node(wi)

        # word_list, graph_word_listを用意
        graph_word_list = important_word_list  # 初期値はimportant_word_list

        # メインのwhileループ
        processed_words = set()  # 処理済みの単語のリスト
        similarity_cache = {}  # 類似度計算のキャッシュ
        iteration_count = 0
        while True:
            new_graph_word_list, processed_words, similarity_cache = extend_graph(wv, G, word_list, graph_word_list,
                                                                                  processed_words, similarity_cache,
                                                                                  sth, N, ih)
            if len(new_graph_word_list) < N or iteration_count >= MAX_ITERATIONS:
                break  # 新たなワードリストの要素数がN未満または最大繰り返し回数に達したら終了
            graph_word_list = new_graph_word_list
            iteration_count += 1

        # 特徴量の計算
        if len(G.nodes) == 0:
            return {
                "number_of_nodes": 0,
                "words_nodes_ratio": 0,
                "cluster_coefficient": np.nan,
                "average_closeness_centrality": np.nan,
                "average_distance": np.nan,
                "diameter": np.nan,
                "density": np.nan,
            }

        # 以下、特徴量計算の処理...
        else:
            cluster_coefficient = nx.average_clustering(G)
            closeness_centrality = nx.closeness_centrality(G)
            average_closeness_centrality = np.mean(list(closeness_centrality.values()))
            degree_centrality = nx.degree_centrality(G)
            betweenness_centrality = nx.betweenness_centrality(G)
            density = nx.density(G)

            try:
                average_distance = nx.average_shortest_path_length(G)
            except nx.NetworkXError:
                try:
                    largest_cc = max(nx.connected_components(G), key=len)
                    largest_subgraph = G.subgraph(largest_cc).copy()
                    average_distance = nx.average_shortest_path_length(largest_subgraph)
                except (ValueError, nx.NetworkXError):
                    average_distance = np.nan

            try:
                diameter = nx.diameter(G)
            except nx.NetworkXError:
                try:
                    largest_cc = max(nx.connected_components(G), key=len)
                    largest_subgraph = G.subgraph(largest_cc).copy()
                    diameter = nx.diameter(largest_subgraph)
                except (ValueError, nx.NetworkXError):
                    diameter = np.nan

            features = {
                "number_of_nodes": len(G.nodes),
                "words_nodes_ratio": len(G.nodes) / len(word_list) if word_list else 0,
                "cluster_coefficient": cluster_coefficient,
                "average_closeness_centrality": average_closeness_centrality,
                "average_distance": average_distance,
                "diameter": diameter,
                "density": density,
            }
    except Exception as e:
        print(f"An error occurred in analyzeNetwork: {e}")
        return {
            "number_of_nodes": 0,
            "words_nodes_ratio": 0,
            "cluster_coefficient": np.nan,
            "average_closeness_centrality": np.nan,
            "average_distance": np.nan,
            "diameter": np.nan,
            "density": np.nan,
        }

    del G
    return features

def load_token_list(path):
    token_list=[]
    with open(path, 'r') as token_file:
        for line in token_file:
            line_list = re.split('\t', line)
            #print('line_list',line_list)
            token_list = token_list + line_list
            #print('token_list', token_list)
    #'\n'を含む場合、それを除外する
    for i in range(len(token_list)):
        token_list[i] = token_list[i].replace("\n",'')
    token_list = [item for item in token_list if item.strip() and item.strip() != '*']
    return token_list


def wvGraphAnalyzer(data_path, part, tf_idf_vectorizer, sth, N, ih):
    os.chdir(data_path)
    dataframe = pd.DataFrame()
    pattern = r'U[A-Z]{2}[0-9]{3}.[1-5].' + str(part) + r'(\.tok|Kana\.tok)'

    for filename in os.listdir(data_path):
        if re.match(pattern, filename):
            file_path = os.path.join(data_path, filename)
            try:
                token_list = load_token_list(file_path)
                features = analyzeNetwork(tf_idf_vectorizer, token_list, sth, N, ih)
                short_pattern = r'U[A-Z]{2}[0-9]{3}.[1-5].' + str(part)
                catch_id = re.findall(short_pattern, filename)[0]
                subid = catch_id[0:6]
                subvisit = catch_id[7]
                subpart = catch_id[9]

                dic = {'ID': subid, 'Visit': subvisit, 'Part': subpart}
                dic.update(**features)
                df = pd.DataFrame(data=dic, index=['val', ])
                dataframe = pd.concat([df, dataframe], axis=0)
            except Exception as e:
                print(f'Error processing file {filename}: {e}')
    return dataframe

def checkFileAnalyzedOrNot(directory_path, dataframe, part):
    # dataframeからID, Visit, Partを読み取り、file_name = "ID-Visit_Part"を作成
    analyzed_file_list = [f"{row['ID']}-{row['Visit']}_{row['Part']}" for index, row in dataframe.iterrows()]
    # 正規表現パターン
    pattern = r'U[A-Z]{2}[0-9]{3}.[1-5].'+ str(part)
    # '.tok' 形式のファイルのパスを格納するリスト
    all_txt_file = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.tok')]
    # ディレクトリ内の全てのファイルのパスを格納するリスト
    all_path = [os.path.join(directory_path, f) for f in os.listdir(directory_path)]
    # all_pathから正規表現にマッチするID部分を抽出し、リスト化
    all_id_list = []
    for path in all_path:
        match = re.search(pattern, path)
        if match:
            all_id_list.append(match.group(0))

    # all_pathに含まれ、all_txt_fileに含まれない要素のみを持つリスト
    rest_path_txt_all = [f for f in all_path if f not in all_txt_file]

    # all_id_listに含まれ、analyzed_file_listに含まれない要素のみを持つリスト
    rest_path_analyzed_all = [id for id in all_id_list if id not in analyzed_file_list]

    print("TXT Files:", all_txt_file)
    print("txtファイル数:", len(all_txt_file))
    # print("All Files:", all_path)
    print("rest files(all-.txt):", rest_path_txt_all)
    print("実際に解析対象となったファイル数:", len(analyzed_file_list))
    print("rest files(all-analyzed):", rest_path_analyzed_all)

    return rest_path_analyzed_all
"""
2024/01/19　w2Vのベクトルからグラフを作成して特徴量を算出
・計算量が多くなるため、どのワードをノードとして取り出すかが重要。
2024/01/26　pip install japanize-matplotlib　グラフの日本語表記
"""

if __name__ == '__main__':
    # Tf-idfモデル作成
    part = 1  # partを指定
    data_path = '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok'
    tf_idf_vectorizer = train_tfidf_model(data_path, part=part)
    # tf_idf_vectorizer = load_tfidf_model(data_path,part)

    # w2v model作成
    model_path = '/home/hnakamura/nlp/w2vGraph/entity_vector/entity_vector.model.txt'
    wv = KeyedVectors.load_word2vec_format(model_path, binary=False)

    # 類似度の閾値と上位N個を取り出すための数
    sth = 0.3  # 例えば 0.5
    N = 10  # 上位N個
    ih = 0  # word2Vecモデルに含まれていない単語についての類似度の代替値

    MAX_ITERATIONS = 10000  # 最大繰り返し回数の定数

    # ここまではグローバル変数

    #　例文解析
    ex_word_list = ["沖縄", "私", "パラダイス", "洗浄", "遊び", "リレー", "カレーライス", "戦争", "ドッキリ", "海", "海水浴", "ダンス",
                 "剣道", "精神", "ラッキー", "悲しい", "困る", "落書き", "シンデレラ", "話",
                 "すごい", "お母さん", "親", "いない", "シンデレラ", "まま", "母", "育てる", "義理", "姉", "愛す"]  # あなたのテキストデータ
    ex_features = analyzeNetwork(tf_idf_vectorizer, ex_word_list, sth, N, ih)  # word_listを解析
    ex_dic = {'ID': 'example', 'Visit': 0, 'Part': 0}
    print("ex_features:", ex_features)
    ex_dic.update(**ex_features)
    ex_df = pd.DataFrame(data=ex_dic, index=['val',])
    print(ex_df)
'''
    # 特徴量
    Eigenvector Centrality

    # 日本語フォントの設定
    # fm.get_font_config()
    #import matplotlib
    import japanize_matplotlib
    # plt.rcParams['font.family'] = 'Hiragino Maru Gothic Pro'
    #matplotlib.rc('font', family='Helvetica')

    # グラフの可視化
    plt.figure(figsize=(12, 12))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_color='skyblue', edge_color='gray', font_family='Hiragino Maru Gothic Pro')
    plt.title("Network Graph")
    plt.show()


    # グラフオブジェクトを削除
    #del G

    # matplotlibの現在の図（figure）をクローズ
    #plt.close('all')
'''


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



TF-IDFモデルがトレーニングされ、'tfidf_model_part1.pkl'として保存されました。
ex_features: {'number_of_nodes': 25, 'words_nodes_ratio': 0.8064516129032258, 'cluster_coefficient': 0.37218181818181817, 'average_closeness_centrality': 0.3813200140547113, 'average_distance': 2.103896103896104, 'diameter': 4, 'density': 0.17666666666666667}
          ID  Visit  Part  number_of_nodes  words_nodes_ratio  \
val  example      0     0               25           0.806452   

     cluster_coefficient  average_closeness_centrality  average_distance  \
val             0.372182                       0.38132          2.103896   

     diameter   density  
val         4  0.176667  


'\n    # 特徴量\n    Eigenvector Centrality\n\n    # 日本語フォントの設定\n    # fm.get_font_config()\n    #import matplotlib\n    import japanize_matplotlib\n    # plt.rcParams[\'font.family\'] = \'Hiragino Maru Gothic Pro\'\n    #matplotlib.rc(\'font\', family=\'Helvetica\')\n\n    # グラフの可視化\n    plt.figure(figsize=(12, 12))\n    pos = nx.spring_layout(G)\n    nx.draw(G, pos, with_labels=True, node_color=\'skyblue\', edge_color=\'gray\', font_family=\'Hiragino Maru Gothic Pro\')\n    plt.title("Network Graph")\n    plt.show()\n\n\n    # グラフオブジェクトを削除\n    #del G\n\n    # matplotlibの現在の図（figure）をクローズ\n    #plt.close(\'all\')\n'

# part 1

In [2]:
# ここから解析・特徴量計算・データフレーム化
part =1
model_path = "/home/hnakamura/nlp/w2vGraph"
tf_idf_vectorizer = load_tfidf_model(model_path,part)
DataFrame = wvGraphAnalyzer(data_path, part, tf_idf_vectorizer, sth, N, ih)
DataFrame.to_csv('wvGraphAnalyzer_part1_R6Apr28_N10_sth0.3.csv')
print(checkFileAnalyzedOrNot(data_path, DataFrame, part))
print("解析から外れたファイル数:", len(checkFileAnalyzedOrNot(data_path, DataFrame, part)))

TF-IDFモデルが'/home/hnakamura/nlp/w2vGraph/tfidf_model_part1.pkl'から読み込まれました。
TXT Files: ['/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UAM019-2_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UBM014-2_3Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UAM010-3_1Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UAM013-5_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UTM015-4_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM008-4_1Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM172-1_3Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM028-1_1Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM173-3_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM020-2_3Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM159-1_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM169-2_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM011-3_2Kana.tok', '/ho

# part2

In [3]:
# Tf-idfモデル作成
part = 2  # partを指定
model_path ="/home/hnakamura/nlp/w2vGraph"
#data_path = '/Users/psychetmdu/Desktop/RecordVoice_tok'
tf_idf_vectorizer = train_tfidf_model(data_path, part=part)

Documents for part 2: ['オペラ 椿姫 なん まず 始め 始まり ある 多分 中世 お話 ん ところ ヴィオレッタ すごい 美しい 女性 いる 方 いう 商館 働く いる そ 毎晩 社交界 きらびやか 社交界 毎晩 お酒 お酒 高話 楽しむ 毎晩 こう きらびやか 日々 送る いる 異端 そこ ある日 １ 人 男性 アルフレード 男性 やってくる ん アルフレード とても こう 誠実 男性 ヴィオレッタ まあ 以前 あなた こと 見る いる ぜひ 私 あなた こと 愛す いる こと 言う ん ヴィオレッタ こう とても 大変 暮らし これ まあ きらびやか 裏 ある こう 社交界 大変 暮らし する 日々 送る くる 人 愛情 簡単 信じる られる 愛す いる 言葉 まあ すぐ こう 浮世 流れ 一緒に わする まあ 忘れる しまう それ 一夜 楽しい いく 日々 毎日 送る それ 私 生き方 ふう 最初 拒む ん こう アルフレード 引く ぜひ 会う できる ふう 言う とき こう とき 珍しい ヴィオレッタ ちょっと 心 動かす れる お花 渡す 返す 戻る 会意 来る くださる ふう 問いかける アルフレード 明日 こと 徐々に ２ 人 心 通う せる いく ん 月日 流れる まあ アルフレード ヴィオレッタ 徐々 ２ 人 お互い 惹く れる 合う 最終的 こう ２ 人 まあ 住む 結婚 一緒 住む 始める ヴィオレッタ 社交界 身 ん 引く 日々 慎ましい 暮らし いう 慎ましい の まあ アルフレード 幸せ 日々 暮らす いる ん アルフレード の すごい ボンボン 坊 財布 財布 事情 あんまり 鑑みる あんまり 知る 人 結構 毎日 暮らし 結構 お金 割く しまう たん ヴィオレッタ こっそり 自分 これ 貯金 切り崩す 自分 金品 これ もらう 金品 自分 持つ てる もの こう 切り売り する 生活 回す いる いう ある まあ アルフレード 黙る こっそり 売る 物 売る 行く する ん それ 最初 アルフレード 知る いる 異端 そのうち 仲居 さん 侍女 いう メイドさん 奥様 そう やる 自分 物 売る いる ん お話 する アルフレード すごい 自分 恥じる 全部 買い戻す 行く 言う 出る ん その間 今度 アル

In [4]:
# ここから解析・特徴量計算・データフレーム化
part = 2
tf_idf_vectorizer = load_tfidf_model(data_path,part)
DataFrame = wvGraphAnalyzer(data_path, part, tf_idf_vectorizer, sth, N, ih)

TF-IDFモデルが'/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/tfidf_model_part2.pkl'から読み込まれました。


In [5]:
# ここから解析・特徴量計算・データフレーム化
part = 2
DataFrame.to_csv('wvGraphAnalyzer_part2_R6Apr28_N10_sth0.3.csv')
print(checkFileAnalyzedOrNot(data_path, DataFrame, part))
print("解析から外れたファイル数:", len(checkFileAnalyzedOrNot(data_path, DataFrame, part)))

TXT Files: ['/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UAM019-2_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UBM014-2_3Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UAM010-3_1Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UAM013-5_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UTM015-4_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM008-4_1Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM172-1_3Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM028-1_1Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM173-3_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM020-2_3Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM159-1_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM169-2_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM011-3_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM187-2_1Kana.tok', '/home/h

# part3

In [6]:
# Tf-idfモデル作成
part = 3  # partを指定
#data_path = '/Users/psychetmdu/Desktop/RecordVoice_tok'
tf_idf_vectorizer = train_tfidf_model(data_path, part=part)

Documents for part 3: ['ある そう そう そう そう もう 何 眠り 話 眠らす 話 お母さん おじいちゃん オバアちゃん 言う くれる はる どんぶり 国庫 どんぶり 国庫 流れる くる の 懐かしい うとうと 膝 上 何 覚える 私 一つ ない 怒る てる ん 私 知る 食べる なる 言う てる の 私 二つ 入る 私 一つ あんた 一つ こっち 食べる ん 知る よい ちゅっ てん かな あんた 取る ん 違う かーっ ちゅ 怒る てる もん ドーナツ 私 ら 終戦 当時 ドーナツ できる ん アメリカ 兵 進駐軍 来る てる 時分 もう すごい 給食 の 今 給食 ご存知 ない かもしれん ある そう 時分 初めて ドーナッツ く つう の 出る くる 昔 私 ら 時分 お父さん 戦争 行く はる お母さん 日雇 いく ちゅ 今 みたい 仕事 ない 団体 掃除 行く 買い出し 行く 京都 田舎 ほう 米 買える 褐炭 着物 持つ みんな それ 米 買う 交換 行く おいも さん もう おいも さん 普通 やる 背負う 帰る 時分 懐かしい Donuts 初めて 食べる 時 はねる びっくり する ん ある 小学校 ３ 年生 時 給食 初めて 出し手 廊下 妹 弟 並ぶ ん そう そう そう 来る 給食 時 先生 何 言う 給食 始まる みんな 入る くる ん 来る お姉ちゃん お兄ちゃん 欲しい 言う 分ける おう 食べる 時代 お話 聞く れる 年 いく 他人 私 小学校 ３ 年生 時 食べる 一つ 足る あんた 取る ん 知る 言う 怒る てる の これ 見事 絵 ん 釣れる てる わー うとうと する てる 魚 釣れる てる 魚 野犬 鴨川 どじょう 救い 行く 懐かしい 思い出す ん 話す はる 年寄り ん ほんま もう 私 ら する もう 憧れる もう * 終戦 こう ずーっと くる ある 凧揚げ とかす これ 家族 * する はる の 車 来店 の 子供 凧揚げ する てる お父さん 本 読む お母さん コーヒー ジュース 飲む ラジオ 持つ くる わんちゃん * おいしい そう なる 思 てる ヨット 懐かしい これ 川遊び 魚釣り 川遊び これ 釣り 緑 木 青々 茂る のどか 風景 もう 大分 戦後 大分 する オバ

In [7]:
# ここから解析・特徴量計算・データフレーム化
part = 3
data_path = '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok' 
tf_idf_vectorizer = load_tfidf_model(data_path,part)
DataFrame = wvGraphAnalyzer(data_path, part, tf_idf_vectorizer, sth, N, ih)
DataFrame.to_csv('wvGraphAnalyzer_part3_R6Apr28_N10_sth0.3.csv')
print(checkFileAnalyzedOrNot(data_path, DataFrame, part))
print("解析から外れたファイル数:", len(checkFileAnalyzedOrNot(data_path, DataFrame, part)))

TF-IDFモデルが'/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/tfidf_model_part3.pkl'から読み込まれました。
TXT Files: ['/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UAM019-2_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UBM014-2_3Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UAM010-3_1Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UAM013-5_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UTM015-4_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM008-4_1Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM172-1_3Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM028-1_1Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM173-3_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM020-2_3Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM159-1_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM169-2_2Kana.tok', '/home/hnakamura/nlp/w2vGraph/RecordVoice_vf_tok/UKM011